##### Bankrate Dataset

In [1]:
# pip install DrissionPage

In [4]:
# bankrate
import pandas as pd
from DrissionPage import ChromiumPage, ChromiumOptions

def download_dataset1(out_file1: str, sample_p: bool = True):
    # Initialize browser options 
    co = ChromiumOptions()
    co.set_headless()
    # Create a browser session
    page = ChromiumPage(co)
    
    # Set a timeout for browser responses
    page.timeout = 2
    
    # URL where the data can be found
    page.get("https://www.bankrate.com/banking/cds/cd-rates/")
    
    # Headers for the DataFrame 
    headers = ['BankName', 'CDTerm', 'APY', 'Min.deposit']
    data_list = []
    
    # Find all div elements that contain rate data
    divs = page.eles("css:div.wrt-RateCard")
    while True:
        # Attempt to load more data
        try:
            page.run_js('document.querySelector("button.wrt-ShowMore-button").click()')
        except:
            break
            
        # Re-fetch the div elements after loading new data
        new_divs = page.eles("css:div.wrt-RateCard")
        
        # Check if new data was loaded
        if len(divs) == len(new_divs):
            break
            
        # If only a sample is needed 
        if sample_p and len(divs) >= 5:
            divs = divs[:5]
            break
    
        divs = new_divs
        
    # Reset the page timeout to the default
    page.timeout = 0
    
    # extract data
    for div in divs:
        try:
            BankName = div.ele("css:p.wrt-RateCard-advertiserLabel").text
        except:
            BankName = div.ele("css:div.wrt-AdvertiserLogo-text").text
            
        # Extract other elements from data 
        CDTerm = div.ele("css:div.wrt-RateCard-body > dl > div:nth-child(3) > dd").text.replace("\n", "")
        APY = div.ele("css:div.wrt-Stat.wrt-Stat--withTooltip > dd > div.wrt-Stat-amount").text + "%"
        Min_deposit = div.ele("css:div.wrt-RateCard-body > dl > div:nth-child(2) > dd").text.replace("\n", "")
        info = [BankName, CDTerm, APY, Min_deposit]
        data_list.append(info)
    
    df = pd.DataFrame(data_list, columns=headers)
    # Determining the filename based on sample_p
    if sample_p:
        filename = f"{out_file1}_samples.csv"
    else:
        filename = out_file1 + '.csv'
    
    df.to_csv(filename, index=False, mode="w", encoding="utf_8_sig")
    print(df.head())

In [5]:
# sample_dataset 
download_dataset1("bankrate", sample_p=True)

                                            BankName CDTerm    APY Min.deposit
0  Third Federal Savings & Loan Association of Cl...    1mo  5.40%        $500
1                                   CBIC Bank USA CD    1yr  5.36%          $0
2                                   CBIC Bank USA CD   13mo  5.36%      $1,000
3                                        Rising Bank    6mo  5.35%      $1,000
4                                        CFG Bank CD    1yr  5.31%        $500


In [3]:
# full dataset
download_dataset1("bankrate", sample_p=False)

                                            BankName CDTerm    APY Min.deposit
0  Third Federal Savings & Loan Association of Cl...    1mo  5.40%        $500
1                                   CBIC Bank USA CD    1yr  5.36%          $0
2                                   CBIC Bank USA CD   13mo  5.36%      $1,000
3                                        Rising Bank    6mo  5.35%      $1,000
4                                        CFG Bank CD    1yr  5.31%        $500


##### Treasury Bills Rates dataset

In [6]:
# Treasury Bills Rates
import requests
import pandas as pd

def download_dataset2(out_file2: str, sample_p: bool = True):
    # Base URL and endpoint for accessing the Treasury Bills data 
    base_url = "https://api.fiscaldata.treasury.gov"
    endpoint = "/services/api/fiscal_service/v1/accounting/od/auctions_query"
    # fields to be retrieved from the API
    fields = "fields=record_date,cusip,security_type,security_term,auction_date,issue_date,maturity_date,price_per100"
    # URL for the API request
    url = f"{base_url}{endpoint}?{fields}&page[size]=100"
    
    # Maximum number of pages to fetch (avoid infinite loops)
    max_pagenum = 100  
    res = requests.get(url)
    # Headers for the DataFrame 
    headers = ['cusip', 'security_type', 'security_term', 'auction_date', 'issue_date', 'maturity_date', 'price_per100']
    data_list = []
    
    # Counter to track the number of pages processed
    page_count = 0
    # processing data as long as the API call is successful and the max page number isn't reached
    while res.status_code == 200 and page_count < max_pagenum:
        data_json = res.json()
         # Extracts the 'data' list from the response
        all_datas = data_json["data"]
        for data in all_datas:
            # Collects information for each record
            info = [data["cusip"], data["security_type"], data["security_term"], data["auction_date"], data["issue_date"], data["maturity_date"], data["price_per100"]]
            data_list.append(info)
            # If sample is needed
            if sample_p and len(data_list) >= 5: 
                break
        if sample_p and len(data_list) >= 5:
            break
        # get the next page URL from the API response
        next_link = data_json["links"].get("next")
        if next_link:
            if not next_link.startswith("http"):
                # concatenation of the next URL
                next_link = f"{base_url}{endpoint}?{next_link}"
#             print("Requesting:", next_link)  
            res = requests.get(next_link)
            page_count += 1
        else:
            break

    df = pd.DataFrame(data_list, columns=headers)
    # Determining the filename based on sample_p
    if sample_p:
        filename = f"{out_file2}_samples.csv"
    else:
        filename = f"{out_file2}.csv"
    
    df.to_csv(filename, index=False, mode="w", encoding="utf_8_sig")
    print(df.head())

In [7]:
# sample_dataset 
download_dataset2("Treasury Bills Rates", sample_p=True)

       cusip security_type security_term auction_date  issue_date  \
0  912827KC5          Note       10-Year   1979-10-31  1979-11-15   
1  912810CK2          Bond       30-Year   1979-11-01  1979-11-15   
2  9127933Y0          Bill       13-Week   1979-12-28  1980-01-03   
3  9127934U7          Bill       26-Week   1979-12-28  1980-01-03   
4  9127935W2          Bill       52-Week   1980-01-02  1980-01-08   

  maturity_date price_per100  
0    1989-11-15         null  
1    2009-11-15         null  
2    1980-04-03         null  
3    1980-07-03         null  
4    1981-01-02         null  


**price_per100** has some null value, in the future analysis in final project, either data imputation tech will be applied to filled the null or those null value will be simply dropped. For now, I will just leave it as it is. 

In [3]:
# full dataset
download_dataset2("Treasury Bills Rates", sample_p=False)

Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=2&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=3&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=4&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=5&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=6&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=7&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnu

Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=58&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=59&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=60&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=61&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=62&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&page%5Bnumber%5D=63&page%5Bsize%5D=100
Requesting: https://api.fiscaldata.treasury.gov/services/api/fiscal_service/v1/accounting/od/auctions_query?&pag

##### Yahoo Finance ETFs Dataset

In [8]:
#  Yahoo Finance ETFs
from DrissionPage import ChromiumPage, ChromiumOptions
import pandas as pd

def download_dataset3(out_file3: str, sample_p=True):
    # Initialize browser options 
    co = ChromiumOptions()
    co.set_headless()
    co.set_page_load_strategy("eager")
    # Create a browser session
    page = ChromiumPage(co)
    
    # Headers for the DataFrame 
    data_list = [['Symbol', 'Name', 'Price', 'Change', '% change', 'Volume', '50 Days Average', '200 Day Average']]
    
    # Loop through pages 
    for idx in range(6): # in this case, the website has 6 pages in total 
        url = f"https://finance.yahoo.com/etfs/?count=100&offset={idx * 100}"
        page.get(url)
        # Select all table row elements 
        trs = page.eles("css:#scr-res-table table > tbody > tr")
        # Iterate over each table row
        for tr in trs:
            td_list = []
            tds = tr.eles("css:td")
            # Extract text from each table
            for td in tds:
                td_list.append(td.text)
            data_list.append(td_list)
            # If only a sample of the data is needed 
            if sample_p and len(data_list) >= 5:
                data_list = data_list[:5]
                break
    # Determining the filename based on sample_p
    if sample_p:
        filename = f"{out_file3}_samples.csv"
    else:
        filename = out_file3 + '.csv'
    df = pd.DataFrame(data_list)
    df.to_csv(filename, header=False, index=False, mode="w", encoding="utf_8_sig")
    print(df.head())

In [9]:
# sample_dataset 
download_dataset3("Yahoo Finance ETFs", sample_p=True)

        0                                                  1      2       3  \
0  Symbol                                               Name  Price  Change   
1    FTCS                   First Trust Capital Strength ETF  81.55    0.00   
2    FPEI  First Trust Institutional Preferred Securities...  17.97    0.00   
3     YLD                    Principal Active High Yield ETF  18.75    0.00   
4    JPST                    JPMorgan Ultra-Short Income ETF  50.34    0.00   

          4        5                6                7     8  
0  % change   Volume  50 Days Average  200 Day Average  None  
1     0.00%  257,150            83.68            78.42        
2     0.00%  237,379            18.11            17.43        
3     0.00%   15,237            18.96            18.54        
4     0.00%   3.832M            50.34            50.22        


In [6]:
# full dataset
download_dataset3("Yahoo Finance ETFs", sample_p=False)

        0                                                  1       2       3  \
0  Symbol                                               Name   Price  Change   
1    KARS  KraneShares Electric Vehicles and Future Mobil...   21.46   +0.35   
2     ECH                             iShares MSCI Chile ETF   26.27   +0.38   
3    CHIQ     Global X MSCI China Consumer Discretionary ETF   17.06   +0.24   
4    KBWP      Invesco KBW Property & Casualty Insurance ETF  101.92   +1.30   

          4        5                6                7     8  
0  % change   Volume  50 Days Average  200 Day Average  None  
1    +1.66%    7,459            22.15            25.76        
2    +1.47%  148,914            26.13            26.90        
3    +1.46%   56,880            16.93            17.82        
4    +1.29%   66,443           102.97            91.41        
